In [1]:
import numpy as np

# from Input_Variables.read_vars import raw_data_storage, \
#                                       mfdfa_wins, mfdfa_q_list, mfdfa_rev_seg, mfdfa_pol_order, \
#                                       fpca_min_time

from Input_Variables.read_vars import raw_data_storage, \
                                      daily_stats_lower, daily_stats_upper

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.daily_stats import Daily_Statistics
# from Feature_Generation.time_series_feature_creation import TS_Feature_Creation

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data(data_location=raw_data_storage)

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features: Daily Summary Statistics
daily_statistics=Daily_Statistics()


# PySpark

### Reading In Data

In [ ]:
####### PySpark
pyspark_df=reading_data.read_in_pyspark()

### Sklearn Pipeline in PySpark

In [ ]:
####### PySpark
pyspark_transform_schema=pandas_udf_data_schema.sklearn_pyspark_schema()
pyspark_transformations=pandas_sklearn_pipeline.pyspark_sklearn_pipeline(df=pyspark_df, 
                                                                         output_schema=pyspark_transform_schema)

### Feature Creation: Daily Stats

# Pandas

### Reading in Data

In [3]:
####### Pandas
pandas_df=reading_data.read_in_pandas()

### Sklearn Pipeline in Pandas

In [4]:
####### Pandas
pandas_transformations=pandas_sklearn_pipeline.pandas_transform_features(df=pandas_df)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements

### Summary Statistics Daily

In [ ]:
pandas_df.columns

In [ ]:
daily_statistics.compressDailyValues_pandas(data=pandas_transformations, lower=daily_stats_lower, upper=daily_stats_upper)

In [ ]:
pandas_transformations.head(2)

### Not Ready, Do Not Run
### Need to add Summary statistics to Features
### Multifractal Detrended Fluctuation Analysis, Poincare Analysis, Functional Principal Component Analysis, Entropy

In [ ]:
ts_feature_creation=TS_Feature_Creation()

# # Issue with fcpaWrapper --> Need to make it loop through every PatientID, also fathon library not working on Mac M1 or M2 chips according to documentation
# mfdfa=ts_feature_creation.do_mfdfa(data=pandas_df.Value.to_numpy(), 
#                                    win_sizes=mfdfa_wins, 
#                                    q_list=mfdfa_q_list, 
#                                    rev_seg=mfdfa_rev_seg, 
#                                    pol_order=mfdfa_pol_order)

# Function Works
poincare=ts_feature_creation.poincare_wrapper(data=pandas_transformations)
print(poincare)

# # Issue with fcpaWrapper --> Need to make it loop through every PatientID and how to define minTime variable
# functional_principal_component_analysis=ts_feature_creation.fpcaWrapper(rawData=pandas_transformations, minTime=fpcs_min_time)

# Fuction Works
entropy=ts_feature_creation.entropy_calculation(data=pandas_transformations)
print(entropy)

### Modeling

### TensorFlow Time Series Probability Models: [TensorFlow](https://www.tensorflow.org/probability/examples/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand)
### PySpark XGBoost: [PySpark](https://xgboost.readthedocs.io/en/stable/tutorials/spark_estimator.html)